# Setting a column based on another one and multiple conditions in pandas

This short notebook shows a way to set the value of one column in a CSV file, that satisfies multiple conditions, by extracting information from another column using the [pandas library](https://pandas.pydata.org/). In addition the standard library [re module](https://docs.python.org/3/library/re.html) will be used for extracting strings using regular expressions.

Setting columns based on other columns is a common task in pandas and there are many straightforward methods for working with numeric data. While working on [ukealong.com](https://ukealong.com/), a website with more than 1,000 play along videos for the ukulele, I had a slightly more complex scenario to deal with. The meta data for the videos, which includes artists and track names is maintained in a CSV file. The source data comes from several YouTube channels, that create these play along videos. Let's look at a small subset of the data, that includes 10 records and only the relevant columns for demonstrating the column setting.

In [1]:
%load_ext signature
import re
import pandas as pd

df = pd.read_csv('data/ukealong-column-setting-sample.csv')
df

,artists,track,title,channel_id
0,NaN,NaN,"""Something"" (The Beatles) Ukulele Play-Along!",UCDglnz22aXMzpug5HbD1bCA
1,NaN,NaN,Arthur Theme Song Ukulele Play-Along!,UCDglnz22aXMzpug5HbD1bCA
2,NaN,NaN,"""Black Magic Woman"" (Santana) Ukulele Play-Along!",UCDglnz22aXMzpug5HbD1bCA
3,Israel Kamakawiwo'ole,White Sandy Beach of Hawaii,"""White Sandy Beach of Hawaii"" (Braddah IZ) Uku...",UCDglnz22aXMzpug5HbD1bCA
4,NaN,What Will We Do With a Drunken Sailor,What Will We Do With a Drunken Sailor,UC6jZYLoYuV1CxY4Stzm6mgg
5,NaN,Frere Jacques,Frere Jacques,UCZjDV_1UEbVsAQA_q9tyTWw
6,NaN,NaN,"""What's Up?"" (4 Non Blondes) Ukulele Play Along!",UCDglnz22aXMzpug5HbD1bCA
7,NaN,NaN,"""Lucy in the Sky with Diamonds"" (The Beatles) ...",UCDglnz22aXMzpug5HbD1bCA
8,NaN,NaN,Lava Ukulele Play-along // Cynthia Lin (Chords...,UCD2q6i-C0ZLJUK-VCp49TJA
9,Weezer,Island In The Sun,"""Island In The Sun"" Ukulele Play-Along!",UCbQn9nS2_W-dsmnr3h_Rpvg


After importing videos from [this playlist](https://www.youtube.com/playlist?list=PLCD6F28C13A0A9AC2) by [Ukulele Underground](https://ukuleleunderground.com/), I faced the task of manually updating the artists and track columns for more than 100 videos. I did this for some of them and thought there must be a better way. Most of their YouTube titles follow the format _"TRACK" (ARTIST) Ukulele Play-Along!_, so the relevant information could be extracted using regular expressions. I only wanted to update records where the `artists` and `track` columns were not already set, which led to the following condition:

In [2]:
condition = (df['channel_id'] == 'UCDglnz22aXMzpug5HbD1bCA') & df['artists'].isna() & df['track'].isna()

The cell below contains the regular expression for matching the value for the `artists` column and the function to extract the name from the title.

In [3]:
re_artists = re.compile(r'\((.+?)\)')

def add_artists(s):
    m = re.search(re_artists, s)
    if m:
        return m.group(1)
    return ''

The next cell contains the regular expression for matching the value for the `track` column and the function to extract the name from the title. Notice that I use the `match` function this time, which tries to match the regular expression at the beginning of string.

In [4]:
re_track = re.compile(r'"(.+?)"')

def add_track(s):
    m = re.match(re_track, s)
    if m:
        return m.group(1)
    return ''

Now the two columns can be updated by applying the respective functions defined above.

In [5]:
df.loc[condition, 'artists'] = df.title.apply(add_artists)
df.loc[condition, 'track'] = df.title.apply(add_track)
df

,artists,track,title,channel_id
0,The Beatles,Something,"""Something"" (The Beatles) Ukulele Play-Along!",UCDglnz22aXMzpug5HbD1bCA
1,,,Arthur Theme Song Ukulele Play-Along!,UCDglnz22aXMzpug5HbD1bCA
2,Santana,Black Magic Woman,"""Black Magic Woman"" (Santana) Ukulele Play-Along!",UCDglnz22aXMzpug5HbD1bCA
3,Israel Kamakawiwo'ole,White Sandy Beach of Hawaii,"""White Sandy Beach of Hawaii"" (Braddah IZ) Uku...",UCDglnz22aXMzpug5HbD1bCA
4,NaN,What Will We Do With a Drunken Sailor,What Will We Do With a Drunken Sailor,UC6jZYLoYuV1CxY4Stzm6mgg
5,NaN,Frere Jacques,Frere Jacques,UCZjDV_1UEbVsAQA_q9tyTWw
6,4 Non Blondes,What's Up?,"""What's Up?"" (4 Non Blondes) Ukulele Play Along!",UCDglnz22aXMzpug5HbD1bCA
7,The Beatles,Lucy in the Sky with Diamonds,"""Lucy in the Sky with Diamonds"" (The Beatles) ...",UCDglnz22aXMzpug5HbD1bCA
8,NaN,NaN,Lava Ukulele Play-along // Cynthia Lin (Chords...,UCD2q6i-C0ZLJUK-VCp49TJA
9,Weezer,Island In The Sun,"""Island In The Sun"" Ukulele Play-Along!",UCbQn9nS2_W-dsmnr3h_Rpvg


After running this code on the actual CSV file most of the Ukulele Underground records showed the correct information in the two columns. Quite frankly, writing the code probably took longer than updating 100 records manually would have, let alone documenting the process in this notebook. On the other hand I can integrate the code in the process of adding new videos when their channel is updated. Moreover, I now have a reference for similar tasks in the future and it hopefully helps other people who face a similar problem.

Last but not least, if you play the ukulele or want to learn it, I hope you'll dig [ukealong.com](https://ukealong.com/), share the site with your friends and show the creators of the play along videos some love by subscribing to their channels and liking their videos on YouTube. Playing music is really a joyful way to spend your time and helps to clear the mind in these times of uncertainty.

In [6]:
%signature